In [1]:
from nnModel import getModel
model = getModel("gz_dev.model")

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [2]:
import re
import shutil
import os
import time
from os.path import isfile, join
from subprocess import Popen

from nnTrainingSession import NNTrainingSession
from nnTrainingFile import createTrainingFiles

currentDir = '.'
gameSourceDir = "D:/GalconZero/Games/"
gameArchiveDirPrefix = "D:/GalconZero/OldGames/Iter"
modelArchiveDir = "D:/GalconZero/OldModels"

NUM_FILES_PER_ITERATION = 100

botProcess = None

session = NNTrainingSession(model)

def restartBots():
    botBasePath = r"C:\Users\Evan Sparano\Documents\GitHub\galcon-alphazero\gbots"
    botProcess = Popen(botBasePath + "\launch8Bots.bat", cwd=botBasePath)

def killBots():
    # TODO: this only kills one bot process? not all of them?
    if botProcess != None:
        os.kill(botProcess.pid, signal.SIGTERM)
    
def archiveGames(iterationNum):
    archiveDir = gameArchiveDirPrefix + str(iterationNum)
    os.makedirs(archiveDir)

    files = os.listdir(gameSourceDir)
    for f in files:
        if 'npz' in f:
            shutil.move(gameSourceDir+f, archiveDir)
        else:
            os.remove(gameSourceDir+f)

def getCurrentIteration():
    onlyFiles = [f for f in os.listdir(modelArchiveDir) if isfile(join(modelArchiveDir, f)) and 'iter' in f]
    return max([ int(re.findall(r'\d+', filename)[0]) for filename in onlyFiles], default=-1) + 1

def trainModel():
    # 10 loops because only a 10th of the data is actually sampled per loop
    # -1 numGamesPerLoop to use all games
    session.doTrain(num_loops=10, numGamesPerLoop=-1, numEpochs=1)
    print("trained the model!")

def nextIteration():
    # TODO: this isn't actually killing the bots, so the bots keep generating unused games while model is training
#     killBots()
    
    prevIterNum = getCurrentIteration()
    model.save(modelArchiveDir + '/gz_dev.iter{0}.model'.format(prevIterNum))
    
    trainModel()
    model.save("gz_dev.model")
    
    archiveGames(prevIterNum)
    
    restartBots()
    
def startTrainingLoop(maxIterations=3):
    restartBots()
    
    while getCurrentIteration() < maxIterations:
        time.sleep(1)
        createTrainingFiles()
        files = os.listdir(gameSourceDir)
        # multiply by two because of pickle and npz files
        if len(files) >= NUM_FILES_PER_ITERATION * 2:
            nextIteration()
            
    killBots()
    

In [ ]:
startTrainingLoop(10000)

Finished loading 90 training files and 10 validation files
Train on 10800 samples, validate on 12000 samples
Epoch 1/1
10800/10800 [==============================] - 4s 397us/step - loss: 5.3982 - policy_loss: 2.3515 - value_loss: 0.3047 - policy_acc: 0.8966 - value_acc: 0.8765 - val_loss: 4.2499 - val_policy_loss: 2.6493 - val_value_loss: 0.1601 - val_policy_acc: 0.7239 - val_value_acc: 0.9295
Finished loading 90 training files and 10 validation files
Train on 10800 samples, validate on 12000 samples
Epoch 1/1
10800/10800 [==============================] - 2s 184us/step - loss: 4.5041 - policy_loss: 2.2743 - value_loss: 0.2230 - policy_acc: 0.8819 - value_acc: 0.9029 - val_loss: 3.9409 - val_policy_loss: 2.6360 - val_value_loss: 0.1305 - val_policy_acc: 0.7164 - val_value_acc: 0.9443
Finished loading 90 training files and 10 validation files
Train on 10800 samples, validate on 12000 samples
Epoch 1/1
10800/10800 [==============================] - 1s 102us/step - loss: 4.0701 - policy_

In [ ]:
from nnTrainingGraph import plotNNTrainingSession

plotNNTrainingSession(session)